# House Prices: Modelos con variable objetivo transformada.

#### *Álvaro Sánchez Castañeda*

En la visualizacion podemos sospechar que una transformación logarítmica de SalePrice podría ser util. Además,el error cuadrático medio de la transformación es igual al RMSLE de la variable sin transformar. De este modo, los modelos se ajustarán usando directamente la metrica propuesta por Kaggle.

**Resumen:**
- Transformación de los datos.
- Entrenamiento y evaluación de modelos.
- Predicciones.

### Inicializamos Spark y cargamos datos y librerias.

In [1]:
import sys

# Añade la ruta de tu carpeta Spark.
spark_path = "C:/Users/AlvaroSanchez91/Apps/spark-2.1.0-bin-hadoop2.7"

sys.path.append(spark_path + '/python')
sys.path.append(spark_path + '/python/lib/py4j-0.10.4-src.zip')

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("nb06") \
    .getOrCreate()



#LIBRERIAS.
import time
import numpy as np
import pandas as pd
from pyspark.sql.functions import col, count, sum as agg_sum
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator as MyEvaluator #Por el log...
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#para evaluator
from pyspark.ml.evaluation import Evaluator
from math import sqrt
from operator import add
from pyspark.sql.functions import avg
from pyspark.sql.functions import log1p, expm1

#DATOS
path = 'C:/Users/AlvaroSanchez91/Desktop/Master Big Data Sevilla/APBD - Arquitecturas y Paradigmas para Big Data/trabajo_final/HousePrices_clean_csv'
full_df = spark.read.csv(path, header=True, inferSchema=True)

#FIJAMOS SEMILLA PARA USARLA POSTERIORMENTE.
global_seed=1

## Preprocesado.

 - ###  Transformaciónes.
 Haremos algunas **transformaciones logaritmicas** (en la exploración grafica de los datos se ve que puede ser conveniente en algunos casos). Tambien transformaremos las variables categoricas a **variables dummies** (esto ya se hizo, pero aquí se muestra otra alternativa mas eficiente). Por último, transformaremos los datos al formato que usan los modelos de la librería **ml**.

In [2]:
##############################
#           LOG1P
##############################

#TRANSFORMACIONES LOGARITMICAS
#Sustituimos las siguientes variables por su transformación.
loglist=['MasVnrArea','OpenPorchSF','BsmtFinSF2','LotArea','1stFlrSF','BsmtUnfSF','BsmtFinSF1','WoodDeckSF']
for c in loglist:
    full_df=full_df.withColumn(c,log1p(full_df[c]))

full_df=full_df.withColumn('SalePrice',log1p(full_df['SalePrice']))
# SEPARACIÓN EN VARIABLES CATEGÓRICAS Y NUMÉRICAS (PREDICTORAS)
tipos=dict(full_df.dtypes)

var_cat=[c for c in tipos if tipos[c]=='string']
var_num=[c for c in tipos if tipos[c]!='string']
#No usamos ni Id ni SalePrice para predecir.
var_num.remove('SalePrice')
var_num.remove('Id')

########################
#        DUMMIES
########################

#Tenemos muchas columnas a transformar, crearemos diccionarios que contengan transformadores y estimadores.

#STRINGINDEXER.
#A cada columna categórica le asociamos un StringIndexer (añadimos _n a las columnas transformadas).
StringIndexers={c:StringIndexer(inputCol=c, outputCol=c+'_n') for c in var_cat}

#ONEHOTENCODER.
#Transforma las salidas de los StringIndexers en variables dummies (añadimos _dummies a las columnas transformadas).
encoders={c:OneHotEncoder(inputCol=c+'_n', outputCol=c+'_dummies') for c in var_cat}

#VECTORASSEMBLER.
#Une las columnas deseadas en una de tipo vector (formato requerido por los modelos de ml).
#Necesitamos crear un vector con las columnas dummies y las variables numéricas originales.
assembler = VectorAssembler(inputCols=[c+'_dummies' for c in var_cat]+var_num, outputCol="features")

#PIPELINE.
#Concatenamos los estimadores anteriores para ajustar y transformar de forma mas sencilla.
#Pasos del pipeline:
piplist=list(StringIndexers.values())+list(encoders.values())+[assembler] 
pip=Pipeline(stages=piplist)
#Ajustamos.
pippre=pip.fit(full_df)
#Con la siguiente orden podemos mostrar la estructura del vector features.
#pippre.transform(full_df).select('features').first().features.toArray()

Ya podemos transformar nuestros datos, teniendolos listos para entrenar modelos de Machine Learning.

In [3]:
#SEPARACION TEST DE KAGGLE.
test_kaggle=full_df.filter(full_df.SalePrice.isNull())
datos=full_df.filter(full_df.SalePrice.isNotNull())
test_kaggle.select('Id').count()

1459

In [4]:
#TRANSFORMACION TEST DE KAGGLE.
test_kaggle_trans=pippre.transform(test_kaggle)
#features contiene variables predictoras, label contiene variable objetivo.
test_kaggle_trans=test_kaggle_trans.select('features',test_kaggle_trans.SalePrice.alias('label'),'Id')
#Necesitamos Id para que kaggle evalue nuestras predicciones.
test_kaggle_trans.show(5)

+--------------------+-----+----+
|            features|label|  Id|
+--------------------+-----+----+
|(260,[0,5,19,26,2...| null|1461|
|(260,[1,8,19,26,2...| null|1462|
|(260,[0,5,19,27,2...| null|1463|
|(260,[0,5,19,25,2...| null|1464|
|(260,[0,7,19,25,2...| null|1465|
+--------------------+-----+----+
only showing top 5 rows



In [5]:
#TRANSFORMACION DATOS DE ENTRENAMIENTO.
datos_trans=pippre.transform(datos)
#features contiene variables predictoras, label contiene variable objetivo.
datos_trans=datos_trans.select('features',datos_trans.SalePrice.alias('label'))
datos_trans.show(5)

+--------------------+------------------+
|            features|             label|
+--------------------+------------------+
|(260,[0,5,19,25,2...| 12.24769911637256|
|(260,[0,6,19,25,3...|12.109016442313738|
|(260,[0,5,19,25,3...|12.317171167298682|
|(260,[0,8,19,27,2...|11.849404844423074|
|(260,[0,5,19,25,3...|12.429220196836383|
+--------------------+------------------+
only showing top 5 rows



- ### Partición train test.
Ya tenemos nuestros datos de entrenamiento y el dato test para que Kaggle evalue nuestras predicciones. Por el momento nos olvidaremos de este conjunto test, y por test nos referiremos a un subconjunto de los datos de entrenamientos que usaremos nosotros para evaluar nuestros modelos.

In [6]:
#PARTICION TRAIN TEST.
train, test = datos_trans.randomSplit([0.8, 0.2], seed=global_seed)

No será necesario estandarizar ahora, los modelos permiten hacerlo internamente.

## Modelos.

- ### Random Forest.

In [7]:
###################################
#   GRID SEARCH
###################################

#MODELO.
rf1 = RandomForestRegressor(predictionCol='pred_rf1', seed=global_seed)
#REJILLA.
paramGrid = ParamGridBuilder() \
    .addGrid(rf1.minInstancesPerNode, [1,20,50,100]) \
    .addGrid(rf1.maxDepth, [5, 7, 10,20]) \
    .addGrid(rf1.numTrees, [10, 50, 100]) \
    .build()
#OPTIMIZAMOS MEDIANTE VALIDACIÓN CRUZADA.
crossval = CrossValidator(estimator=rf1,
                          estimatorParamMaps=paramGrid,
                          evaluator=MyEvaluator(predictionCol='pred_rf1'),
                          numFolds=3, 
                          seed=global_seed) 
t0 = time.time()
rf1_cv_model = crossval.fit(train)
tt = time.time() - t0
print('Tiempo ajuste de parámetros {} min.'.format(round(tt/60,2)))

#TABLA CON RESULTADOS
par_map = rf1_cv_model.getEstimatorParamMaps()
lpars = [{par.name: value for par, value in par_comb.items()} for par_comb in par_map]
pars_df = pd.DataFrame(lpars)
pars_df['score'] = rf1_cv_model.avgMetrics
pars_df.sort_values(by='score', ascending=False)

Tiempo ajuste de parámetros 8.6 min.


,maxDepth,minInstancesPerNode,numTrees,score
47,20,100,100,0.207565
46,10,100,100,0.207565
45,7,100,100,0.207565
44,5,100,100,0.207565
43,20,100,50,0.207477
42,10,100,50,0.207477
41,7,100,50,0.207477
40,5,100,50,0.207477
36,5,100,10,0.206376
39,20,100,10,0.206376


In [8]:
###############################################
#        MODELO CON MEJORES PARÁMETROS
###############################################

#EVALUAMOS MEDIANTE TEST.
test=rf1_cv_model.bestModel.transform(test)
test.show(5)
evaluator=MyEvaluator(predictionCol='pred_rf1')
modelos_eval=pd.DataFrame([('rf1',evaluator.evaluate(test))],columns=['Modelo','Score'])
modelos_eval

+--------------------+------------------+------------------+
|            features|             label|          pred_rf1|
+--------------------+------------------+------------------+
|(260,[0,5,19,25,2...|12.211065162153215|12.306654304532655|
|(260,[0,5,19,25,2...|11.648373769145277|11.597927404368045|
|(260,[0,5,19,25,2...|11.891368749982085| 12.03153311685902|
|(260,[0,5,19,25,2...|12.066241450757124| 12.09600255335475|
|(260,[0,5,19,25,2...|11.976665770490767|11.933986288710686|
+--------------------+------------------+------------------+
only showing top 5 rows



,Modelo,Score
0,rf1,0.134972


- ### Gradient Boosting.

In [9]:
###################################
#   GRID SEARCH
###################################

#MODELO.
gbt = GBTRegressor(predictionCol='pred_gbt',featuresCol='features', seed=global_seed)
#REJILLA.
paramGrid = ParamGridBuilder() \
    .addGrid(gbt.minInstancesPerNode , [20,25,30,40]) \
    .addGrid(gbt.maxDepth, [2,3,4]) \
    .addGrid(gbt.maxIter , [50,100]) \
    .build()
    
#OPTIMIZAMOS MEDIANTE VALIDACIÓN CRUZADA.
crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=MyEvaluator(predictionCol='pred_gbt'),
                          numFolds=3, 
                          seed=global_seed) 
crossval
t0 = time.time()
gbt_cv_model = crossval.fit(train)
tt = time.time() - t0
print('Tiempo ajuste de parámetros {} min.'.format(round(tt/60,2)))

#TABLA CON RESULTADOS.
par_map = gbt_cv_model.getEstimatorParamMaps()
lpars = [{par.name: value for par, value in par_comb.items()} for par_comb in par_map]
pars_df = pd.DataFrame(lpars)
pars_df['score'] = gbt_cv_model.avgMetrics
pars_df.sort_values(by='score', ascending=False)

Tiempo ajuste de parámetros 26.23 min.


,maxDepth,maxIter,minInstancesPerNode,score
10,4,50,25,0.172234
4,4,50,20,0.172079
16,4,50,30,0.171563
22,4,50,40,0.169684
14,3,50,30,0.169177
20,3,50,40,0.166522
8,3,50,25,0.166465
11,4,100,25,0.166368
5,4,100,20,0.165947
2,3,50,20,0.164092


In [10]:
###############################################
#        MODELO CON MEJORES PARÁMETROS
###############################################

#SCORE EN TEST.
test=gbt_cv_model.bestModel.transform(test)
test.show(5)
evaluator=MyEvaluator(predictionCol='pred_gbt')
modelos_eval=pd.concat([modelos_eval,pd.DataFrame([('gbt',evaluator.evaluate(test))],columns=['Modelo','Score'])])
modelos_eval

+--------------------+------------------+------------------+------------------+
|            features|             label|          pred_rf1|          pred_gbt|
+--------------------+------------------+------------------+------------------+
|(260,[0,5,19,25,2...|12.211065162153215|12.306654304532655|12.223631817531988|
|(260,[0,5,19,25,2...|11.648373769145277|11.597927404368045|11.503183030142388|
|(260,[0,5,19,25,2...|11.891368749982085| 12.03153311685902| 11.97013986377167|
|(260,[0,5,19,25,2...|12.066241450757124| 12.09600255335475|12.017246809363451|
|(260,[0,5,19,25,2...|11.976665770490767|11.933986288710686| 11.98347843200379|
+--------------------+------------------+------------------+------------------+
only showing top 5 rows



,Modelo,Score
0,rf1,0.134972
0,gbt,0.140336


- ### Elastic Net.

In [11]:
###################################
#   GRID SEARCH
###################################

#MODELO.
lr = LinearRegression(predictionCol='pred_lr',featuresCol='features')

#REJILLA.
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam , [0, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2]) \
    .addGrid(lr.elasticNetParam, [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.8, 0.9]) \
    .build()

#OPTIMIZAMOS MEDIANTE VALIDACIÓN CRUZADA.
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=MyEvaluator(predictionCol='pred_lr'),
                          numFolds=3, 
                          seed=global_seed) 
t0 = time.time()
lr_cv_model = crossval.fit(train)
tt = time.time() - t0
print('Tiempo ajuste de parámetros {} min.'.format(round(tt/60,2)))

#TABLA CON RESULTADOS.
par_map = lr_cv_model.getEstimatorParamMaps()
lpars = [{par.name: value for par, value in par_comb.items()} for par_comb in par_map]
pars_df = pd.DataFrame(lpars)
pars_df['score'] = lr_cv_model.avgMetrics
pars_df.sort_values(by='score', ascending=False)

Tiempo ajuste de parámetros 7.35 min.


,elasticNetParam,regParam,score
62,0.90,0.200,0.291022
55,0.80,0.200,0.275892
61,0.90,0.150,0.252392
54,0.80,0.150,0.241329
60,0.90,0.100,0.214193
53,0.80,0.100,0.206314
48,0.30,0.200,0.195155
41,0.25,0.200,0.185947
47,0.30,0.150,0.179073
21,0.15,0.000,0.178549


In [12]:
###############################################
#        MODELO CON MEJORES PARÁMETROS
###############################################

#SCORE EN TEST.
test=test.drop('pred_lr')
test=lr_cv_model.bestModel.transform(test)
evaluator=MyEvaluator(predictionCol='pred_lr')
evaluator.evaluate(test)
modelos_eval=pd.concat([modelos_eval,pd.DataFrame([('lr',evaluator.evaluate(test))],columns=['Modelo','Score'])])
modelos_eval

,Modelo,Score
0,rf1,0.134972
0,gbt,0.140336
0,lr,0.126745


- ### Ensamblado de modelos.

In [13]:
#PREDECIMOS SOBRE TRAIN.
train=gbt_cv_model.bestModel.transform(train)
train=lr_cv_model.bestModel.transform(train)
train=rf1_cv_model.bestModel.transform(train)
predassembler = VectorAssembler(inputCols=['pred_lr','pred_gbt'], outputCol="features_asembler")

#TRANSFORMAMOS EN VECTOR.
train=predassembler.transform(train)
test=predassembler.transform(test)


#COMBINACIÓN LINEAL MODELOS.
lr_asembler = LinearRegression(predictionCol='pred_asembler',featuresCol='features_asembler')
lr_asembler_model=lr_asembler.fit(train)
print(lr_asembler_model.coefficients)

#EVALUAMOS EN TEST.
test=lr_asembler_model.transform(test)
evaluator=MyEvaluator(predictionCol='pred_asembler')
modelos_eval=pd.concat([modelos_eval,pd.DataFrame([('assembling',evaluator.evaluate(test))],columns=['Modelo','Score'])])

modelos_eval

[0.269151081543,0.766988902062]


,Modelo,Score
0,rf1,0.134972
0,gbt,0.140336
0,lr,0.126745
0,assembling,0.131187


- ### Model Stacking.

In [14]:
#PARTICION TRAIN EN DOS CONJUNTOS.
train1, train2 = train.randomSplit([0.5, 0.5], seed=global_seed)

###########################################
#ENTRENAMOS LOS MODELOS CON train1.
###########################################

#ELASTIC NET
lr_ms = LinearRegression(predictionCol='pred_lr_ms',featuresCol='features',elasticNetParam=0.25,regParam=0.2)
lr_ms_model=lr_ms.fit(train1)
train2=lr_ms_model.transform(train2)
test=lr_ms_model.transform(test)

#GRADIENT BOOSTING
gbt_ms = GBTRegressor(predictionCol='pred_gbt_ms',featuresCol='features',maxDepth=2,maxIter=100,minInstancesPerNode=30, seed=global_seed)
gbt_ms_model=gbt_ms.fit(train1)
train2=gbt_ms_model.transform(train2)
test=gbt_ms_model.transform(test)

#RANDOM FOREST.
rf_ms = RandomForestRegressor(predictionCol='pred_rf1_ms',numTrees=50,maxDepth=20, seed=global_seed)
rf_ms_model=rf_ms.fit(train1)
train2=rf_ms_model.transform(train2)
test=rf_ms_model.transform(test)

#UNIMOS PRED A FEATURES.
ms_assembler = VectorAssembler(inputCols=['pred_lr_ms','pred_gbt_ms','pred_rf1_ms','features'], outputCol="features_ms")
train2=ms_assembler.transform(train2)
test=ms_assembler.transform(test)

###########################################
#ENTRENAMOS MODELO FINAL CON train2.
###########################################

#MODELO.
gbt_wrapper = GBTRegressor(predictionCol='pred_ms',featuresCol='features_ms', seed=global_seed)
#REJILLA.
paramGrid = ParamGridBuilder() \
    .addGrid(gbt_wrapper.minInstancesPerNode , [20,25,30,40]) \
    .addGrid(gbt_wrapper.maxDepth, [2,3,4]) \
    .addGrid(gbt_wrapper.maxIter , [50,100]) \
    .build()
    
#OPTIMIZAMOS MEDIANTE VALIDACIÓN CRUZADA.
crossval = CrossValidator(estimator=gbt_wrapper,
                          estimatorParamMaps=paramGrid,
                          evaluator=MyEvaluator(predictionCol='pred_ms'),
                          numFolds=3, 
                          seed=global_seed) 
crossval
t0 = time.time()
gbt_wrapper_model = crossval.fit(train2)
tt = time.time() - t0
print('Tiempo ajuste de parámetros {} min.'.format(round(tt/60,2)))

#TABLA CON RESULTADOS.
par_map = gbt_wrapper_model.getEstimatorParamMaps()
lpars = [{par.name: value for par, value in par_comb.items()} for par_comb in par_map]
pars_df = pd.DataFrame(lpars)
pars_df['score'] = gbt_cv_model.avgMetrics
pars_df.sort_values(by='score', ascending=False)

Tiempo ajuste de parámetros 89.72 min.


,maxDepth,maxIter,minInstancesPerNode,score
10,3,50,40,0.172234
4,3,50,25,0.172079
16,3,100,25,0.171563
22,3,100,40,0.169684
14,4,100,20,0.169177
20,4,100,30,0.166522
8,4,50,30,0.166465
11,4,50,40,0.166368
5,4,50,25,0.165947
2,4,50,20,0.164092


In [15]:
#EVALUAMOS EN TEST.
test=gbt_wrapper_model.bestModel.transform(test)
evaluator=MyEvaluator(predictionCol='pred_ms')
evaluator.evaluate(test)
modelos_eval=pd.concat([modelos_eval,pd.DataFrame([('MS',evaluator.evaluate(test))],columns=['Modelo','Score'])])
modelos_eval

,Modelo,Score
0,rf1,0.134972
0,gbt,0.140336
0,lr,0.126745
0,assembling,0.131187
0,MS,0.140013


### Submissions.
 Hemos evaluado distintos modelos, lo conveniente sería escoger los mejores y reentrenarlos usando los mejores parámetros (según lo aquí visto). De momento nos limitaremos a usar los ya construidos para predecir sobre los datos **test_kaggle** y ver que resultados nos da la plataforma.

In [ ]:
#######################################
#        ENSAMBLADO
#######################################

#PREDECIMOS MEDIANTE GBT Y LR.
test_kaggle_trans=gbt_cv_model.bestModel.transform(test_kaggle_trans)
test_kaggle_trans=lr_cv_model.bestModel.transform(test_kaggle_trans)

#TRANSFORMAMO SEN VECTOR.
test_kaggle_trans=predassembler.transform(test_kaggle_trans)

#COMBINACIÓN LINEAL DE PREDICCIONES.
test_kaggle_trans=lr_asembler_model.transform(test_kaggle_trans)

#TRAEMOS LOS DATOS A LOCAL Y GUARDAMOS EN CSV.
#Esto lo podemos hacer por que no estamos ante un verdadero problema Big Data.
submission=test_kaggle_trans.drop('SalePrice').select('Id',expm1(test_kaggle_trans['pred_asembler']).alias('SalePrice'))
local=submission.collect()
local=pd.DataFrame(local,columns=['Id','SalePrice'])
local.to_csv('submission_asembler_log.csv',index=False)

#######################################
#        MODEL STACKING
#######################################

#ELASTIC NET, GRADIENT BOOSTING Y RANDOM FOREST.
test_kaggle_trans=lr_ms_model.transform(test_kaggle_trans)
test_kaggle_trans=gbt_ms_model.transform(test_kaggle_trans)
test_kaggle_trans=rf_ms_model.transform(test_kaggle_trans)

#UNIMOS PRED A FEATURES.
test_kaggle_trans=ms_assembler.transform(test_kaggle_trans)

#MODELO WRAPPER.
test_kaggle_trans=gbt_wrapper_model.bestModel.transform(test_kaggle_trans)

#TRAEMOS LOS DATOS A LOCAL Y GUARDAMOS EN CSV.
#Esto lo podemos hacer por que no estamos ante un verdadero problema Big Data.
submission=test_kaggle_trans.drop('SalePrice').select('Id',expm1(test_kaggle_trans['pred_ms']).alias('SalePrice'))
local=submission.collect()
local=pd.DataFrame(local,columns=['Id','SalePrice'])
local.to_csv('submission_ms_log.csv',index=False)

############################
#          LR
############################

#TRAEMOS LOS DATOS A LOCAL Y GUARDAMOS EN CSV.
#Esto lo podemos hacer por que no estamos ante un verdadero problema Big Data.
submission=test_kaggle_trans.drop('SalePrice').select('Id',expm1(test_kaggle_trans['pred_lr']).alias('SalePrice'))
local=submission.collect()
local=pd.DataFrame(local,columns=['Id','SalePrice'])
local.to_csv('submission_lr_log.csv',index=False)

In [ ]:
#spark.stop()